In [1]:
import os

In [2]:
%pwd

'c:\\Users\\hp26\\Documents\\Text_Summarization_project\\Research'

In [3]:
os.chdir("../")
%pwd

'c:\\Users\\hp26\\Documents\\Text_Summarization_project'

entity file

In [4]:
# getting information from config/config.yaml

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir = Path
    data_path = Path
    tokenizer_name = Path

configuration file

In [6]:
from textSummarizer.constrants import *
from textSummarizer.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self, 
                config_filepath: CONFIG_FILE_PATH, 
                params_filepath: PARAMS_FILE_PATH ):
        self.config_filepath = read_yaml(config_filepath)
        self.params_filepath =read_yaml( params_filepath)

        create_directories( self.config.artifacts_root)


    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            tokenizer_name = config.tokenizer_name,
        )
        return data_transformation_config


components file


In [7]:
import os
from textSummarizer.logging import logger
from datasets import load_dataset, load_from_disk
from transformers import AutoTokenizer

class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)


    def convert_examples_to_features(self,example_batch):
        input_encoding = self.tokenizer(example_batch['dialogue'],max_length=1024,truncation =True)

        with self.tokenizer.as_target_tokenizer():
            target_encoding = self.tokenizer(example_batch['summary'],max_length=128,truncation =True)

        return{
            'input_ids': input_encoding['input_ids'],
            'attention_mask': input_encoding['attention_mask'],
            'labels': target_encoding['input_ids']
            
        }
    
    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched=True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))

pipeline file

In [8]:
try:
    config =ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config = data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e


TypeError: ConfigurationManager.__init__() missing 2 required positional arguments: 'config_filepath' and 'params_filepath'